##  Snake BFS Solver 

In [1]:
from base_apple_snake import Base, Apple, Snake
from bfs_mixed_strategy import BFS, Mixed 
from dataclasses import dataclass

import pygame
import cv2
import numpy as np


import time
import sys

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
DARKGRAY = (40, 40, 40)

pygame.init()
clock = pygame.time.Clock()
font = pygame.font.SysFont('Helvetica', 14)


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
@dataclass
class SnakeGame(Base):
    fps: int = 40 ## 20 ## 10 ## 60

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.kwargs = kwargs

        pygame.init()
        self.clock = pygame.time.Clock()
        ## 0.25 of display used for the snake
        ## 0.25 of display square used for info
        ## 0.25 of display square used for BFS path snake-head <--> apple (or, sometimes, head <--> tail)
        self.display = pygame.display.set_mode((self.window_width*2, self.window_height*2))
        pygame.display.set_caption('Snake bfs solver')
        
        self.max_len_of_snake = self.cell_width * self.cell_height
        self.go_to_smth = 0
        self.start_game = 0
        self.time_of_game = 0

    def game(self):
        snake = Snake(**self.kwargs)

        apple = Apple(**self.kwargs)
        apple.refresh(snake=snake)

        step_time = []
        runGame = True
        
        self.start_game = time.time() 

        while runGame:
            for event in pygame.event.get():  # event handling loop
                if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
                    runGame = False
                    
            keys = pygame.key.get_pressed()
            if keys[pygame.K_r]:
                paused = True
                while paused == True:
                    clock.tick(10)
                    pygame.event.pump()
                    for event in pygame.event.get():
                        if event.type == pygame.KEYDOWN:
                            paused = False               

            start_time = time.time()

            # BFS Solver
            self.strategy = Mixed(snake=snake, apple=apple, **self.kwargs)
            new_head, bfs_path, self.go_to_smth = self.strategy.run_mixed()   
            
            end_time = time.time()
            move_time = end_time - start_time
            step_time.append(move_time)

            ## print('game: head', new_head)
            snake.move(new_head=new_head, apple=apple)

            if snake.is_dead:
                print(snake.body)
                print("Dead")
                break
            elif snake.eaten:
                apple.refresh(snake=snake)

            if snake.score + snake.initial_length >= self.max_len_of_snake:
                break

            self.display.fill(BLACK)
            self.draw_panel()
            self.draw_snake(snake.body)
            if bfs_path is not None:
                self.time_of_game = time.time() - self.start_game
                self.draw_bfs_path(bfs_path)

            self.draw_apple(apple.location)
            pygame.display.update()
            self.clock.tick(self.fps)
            
        print(f"Score: {snake.score}")
        print(f"Mean step time: {self.mean(step_time)}")
        

    def draw_snake(self, snake_body):
        for snake_block_x, snake_block_y in snake_body:
            x = snake_block_x * self.cell_size
            y = snake_block_y * self.cell_size
            snake_block = pygame.Rect(x, y, self.cell_size - 1, self.cell_size - 1)
            pygame.draw.rect(self.display, WHITE, snake_block)

        # Draw snake's head
        x = snake_body[-1][0] * self.cell_size
        y = snake_body[-1][1] * self.cell_size
        snake_block = pygame.Rect(x, y, self.cell_size - 1, self.cell_size - 1)
        pygame.draw.rect(self.display, GREEN, snake_block)

        # Draw snake's tail
        x = snake_body[0][0] * self.cell_size
        y = snake_body[0][1] * self.cell_size
        snake_block = pygame.Rect(x, y, self.cell_size - 1, self.cell_size - 1)
        pygame.draw.rect(self.display, BLUE, snake_block)
        
        ### **** Print Length *****
        len_of_snake = len(snake_body)
        lensnaketext = font.render('          LEN OF SNAKE: ' + str(len_of_snake), False, (255, 255, 255))
        maxsnaketext = font.render('          MAX OF SNAKE: ' + str(self.max_len_of_snake), False, (255, 255, 255))
        tt = round(self.time_of_game,2)
        timegametext = font.render('          TIME OF GAME: ' + str(tt), False, (255, 255, 255))
        
        x_pos= (int)(1.2*self.window_width)
        self.display.blit(maxsnaketext, (x_pos, 40))
        self.display.blit(lensnaketext, (x_pos, 80))
        self.display.blit(timegametext, (x_pos, 120))

    def draw_apple(self, apple_location):
        apple_x, apple_y = apple_location
        apple_block = pygame.Rect(apple_x * self.cell_size, apple_y * self.cell_size, self.cell_size, self.cell_size)
        pygame.draw.rect(self.display, RED, apple_block)

    def draw_panel(self):
        for x in range(0, self.window_width, self.cell_size):  # draw vertical lines
            pygame.draw.line(self.display, DARKGRAY, (x, 0), (x, self.window_height))
        for y in range(0, self.window_height, self.cell_size):  # draw horizontal lines
            pygame.draw.line(self.display, DARKGRAY, (0, y), (self.window_width, y))
        
        pygame.draw.line(self.display, WHITE, (0, self.window_height), (self.window_width*2, self.window_height))
        pygame.draw.line(self.display, WHITE, (self.window_width, 0), (self.window_width, self.window_height*2))
            
    def draw_bfs_path(self, path):
        ind = 0
        for elem in path:
                           
            ## print('path coord: ', elem[0], elem[1])
            x = elem[0] * self.cell_size
            y = elem[1] * self.cell_size + self.window_height
                        
            path_block = pygame.Rect(x, y, self.cell_size - 1, self.cell_size - 1)
            
            if ind+1 == len(path):
                if self.go_to_smth == self.strategy.GO_APPLE:    
                    pygame.draw.rect(self.display, RED, path_block)
                if self.go_to_smth == self.strategy.GO_TAIL:    
                    pygame.draw.rect(self.display, BLUE, path_block)
            else:
                if ind > 0:
                    pygame.draw.rect(self.display, CYAN, path_block)
                    
            if ind == 1:        
                # Draw snake's head
                pygame.draw.rect(self.display, GREEN, path_block)
                                
            ind += 1
                                

In [3]:
SnakeGame().game()
gameExit = True
while gameExit:
    clock.tick(100)
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
            pygame.quit()
            gameExit = False        

Score: 193
Mean step time: 0.0009
